# I. Downloading Data

### Importing Libraries

In [712]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

### Accessing API

In [713]:
scope = ['https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('sanitized-data-502a805415d7.json',scope)
client = gspread.authorize(credentials)
sheet = client.open('da-task').sheet1

### Accessing Data

In [714]:
df = pd.DataFrame(sheet.get_all_records())

### Reading Data

In [715]:
df.head()

,created_time,full_name,email,phone_number,city
0,11/3/21,Lakshminarayana Murthy Ganti,lnmganti@yahoo.com,919483163948,Bangalore
1,11/3/21,👰,u.deglurkar@gmail.com,919823010974,Pune
2,1/2/21,Preeti Kumari,INDIA@MMTZ.COM,1507490000000000.0,Gurugram
3,11/3/21,Ganesh Viswanathan 👰photo,gansgans@rediffmail.com✶,608645796670595,Bangalore
4,11/3/21,ಬಸವರಾಜ,take2video1@gmail.com,1384930000000000.0,Mumbai


### Looking into columns of our Data

In [716]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   created_time  25 non-null     object
 1   full_name     25 non-null     object
 2   email         25 non-null     object
 3   phone_number  25 non-null     object
 4   city          25 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


# II. Data Cleaning

## Errors to resolve

### Date
DD MM YYYY
Remove Any invalid data

### Full Name
Split in First Name and List Name
First Letter in Uppercase in First Name and Last Name
If there is no Last Name, Add First Name as Last Name
If No Valid name, write "No Name Found"

### Email
Make it lower case
Must be a valid email
Leave blank if there is no valid email
Remove any invalid data

### Phone
"Format : Country Code and Phone Number 
918800302222"
Remove any invalid data
If there is any international data, highlight that by color
If country code is mission, add 91 before number

### Dashboard
Please create dashboard in Google sheet 
Create Charts, Graphs what you feel is the best 
You can add some dummy data if you think data is less


In [717]:
df.created_time

0          11/3/21
1          11/3/21
2           1/2/21
3          11/3/21
4          11/3/21
5       2 jan 2021
6          11/3/21
7          11/3/21
8          11/2/21
9     Jan 22, 1920
10         11/2/21
11         11/2/21
12         11/2/21
13         11/2/21
14    4th Apr 1992
15         11/2/21
16         11/2/21
17         11/1/21
18         11/1/21
19         11/1/21
20         11/1/21
21         11/1/21
22         11/1/21
23         11/1/21
24         11/1/21
Name: created_time, dtype: object

In [718]:
df.created_time.unique()

array(['11/3/21', '1/2/21', '2 jan 2021', '11/2/21', 'Jan 22, 1920',
       '4th Apr 1992', '11/1/21'], dtype=object)

### Date

In [719]:
df.created_time = df.created_time.str.replace(' /','/').str.replace('/ ','/')
df.created_time = df.created_time.str.replace('jan','/1/').str.replace('Apr','4')
df.created_time = df.created_time.str.replace('2021','21')
df.created_time = df.created_time.str.replace(',','/')
df.created_time = df.created_time.str.replace('th','').str.replace('Jan','1')
df.created_time = df.created_time.str.replace(" ",'/')
df.created_time = df.created_time.str.replace('/1/','/01/').str.replace('/2/','/02/').str.replace('/3/','/03/').str.replace('/4/','/04/').str.replace('/21','/2021')
df.created_time = df.created_time.str.replace('//','/')


### Name

#### First Name

In [720]:
first_name = df.full_name.str.replace(' 👰photo','').str.replace('👰','').str.replace('🖤','').str.split(" ").str[0:2].str.join(' ').str.capitalize()
df['First Name'] = first_name

#### Last Name

In [721]:
last_name = df.full_name.str.replace(' 👰photo','').str.replace('👰','').str.replace('🖤','').str.split(" ").str[-1].str.capitalize()
df['Last Name'] = last_name

### Email

In [722]:
email_id = df.email.str.lower().str.replace('🎀','').str.replace('@gmail','gmail').str.replace('gmail','@gmail').str.replace('✶','')
df['Email ID'] = email_id

### Phone Number

In [723]:
df.phone_number = df.phone_number.astype(str).str.replace('-','').str.replace('AC','').str.replace('0.','91')

C:\Users\wazid\AppData\Local\Temp/ipykernel_8872/3029647224.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.phone_number = df.phone_number.astype(str).str.replace('-','').str.replace('AC','').str.replace('0.','91')


In [724]:
values = []
for y in df['phone_number']:
    if len(y)==12:
        values.append(y)
    else:
        values.append('NaN')
        
df['Phone Number'] = values

### City

In [725]:
df['City'] = df.city.str.capitalize()

### Dropping unecessary features

In [727]:
df = df.drop(['full_name','email','phone_number','city'],axis=1)

### Importing cleaned data

In [742]:
df.to_excel("Sanitized_data.xlsx")